In [1]:
import os
import getpass

# Prompt the user for the API key
api_key = getpass.getpass("Enter your LANGSMITH API Key: ")

# Set environment variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = api_key

# You can now use these environment variables in your code
print("LANGSMITH_TRACING:", os.getenv("LANGSMITH_TRACING"))
print("LANGSMITH_API_KEY has been set.")

Enter your LANGSMITH API Key: ··········
LANGSMITH_TRACING: true
LANGSMITH_API_KEY has been set.


In [2]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [3]:
ClaudeapiKey = getpass.getpass("Enter your Claude API key: ")

Enter your Claude API key: ··········


In [4]:
!pip install langsmith openai langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existi

## Model 1: gpt-4o-mini

In [5]:
from langsmith import traceable, wrappers
from openai import Client

openai = wrappers.wrap_openai(Client())

MODEL = "gpt-4o-mini"

from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=MODEL,
    max_tokens=200
)

@traceable(name="gpt_4o_mini_tracing")
def get_sentiment(messages):
    return llm.invoke(messages).content

def build_input_prompt(statement):
    user_content = f"""You will be provided a statement from twitter and your job is to determine sentiments of it. Sentiments should be below and just give sentiment as response.
    sentiments: ["sadness", "happiness", "neutral"]

    Here is the statement.
    <statement>{statement}</statement>

    What is the sentiment of statement? Please respond with a only one word. e.g. sadness"""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

<ipython-input-5-72a4a90f2f15>:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [6]:
from langsmith import Client

client = Client()
dataset_name = "Sentiment Dataset"


In [7]:
from langsmith.schemas import Example, Run

def correct_label(root_run: Run, example: Example) -> dict:
    score = root_run.outputs.get("output") == example.outputs.get("golden_answer")
    return {"score": int(score), "key": "correct_label"}

In [8]:
from langsmith.evaluation import evaluate


results = evaluate(
    lambda inputs: get_sentiment(build_input_prompt(inputs["input"])),
    data=dataset_name,
    evaluators=[correct_label],
    experiment_prefix="Sentiment",
)

View the evaluation results for experiment: 'Sentiment-8d8aea0a' at:
https://smith.langchain.com/o/9c75e871-1666-51ff-aaa8-7c3e38165404/datasets/71866eb4-2615-43be-80da-05678ed760ec/compare?selectedSessions=534050e9-11be-4d65-a085-d7e8eef85eab




0it [00:00, ?it/s]

## Model 2: claude-3-sonnet-20240229

In [9]:
from langsmith import traceable, wrappers
from openai import Client
import urllib
import json

openai = wrappers.wrap_openai(Client())

MODEL = "claude-3-sonnet-20240229"

def get_sentiment(messages):
    """Make an API call to the Anthropic Claude model."""
    url = "https://api.anthropic.com/v1/messages"
    headers = {
        'x-api-key': ClaudeapiKey,
        'anthropic-version': '2023-06-01',
        'content-type': 'application/json'
    }
    data = json.dumps({
        "model": "claude-3-sonnet-20240229",
        "max_tokens": 200,
        "messages": messages
    }).encode('utf-8')

    req = urllib.request.Request(url, data=data, headers=headers, method='POST')

    try:
        with urllib.request.urlopen(req) as response:
            output = response.read()
            return json.loads(output)['content'][0]['text']
    except urllib.error.HTTPError as e:
        print(f"HTTP error: {e.code} - {e.reason}")
        print(f"Response body: {e.read().decode()}")

@traceable(name="claude_3_sonnet_20240229_tracing")
def get_sentiment(messages):
    return llm.invoke(messages).content

def build_input_prompt(statement):
    user_content = f"""You will be provided a statement from twitter and your job is to determine sentiments of it. Sentiments should be below and just give sentiment as response.
    sentiments: ["sadness", "happiness", "neutral"]

    Here is the statement.
    <statement>{statement}</statement>

    What is the sentiment of statement? Please respond with a only one word. e.g. sadness"""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

In [10]:
from langsmith import Client

client = Client()
dataset_name = "Sentiment Dataset"


In [11]:
from langsmith.schemas import Example, Run

def correct_label(root_run: Run, example: Example) -> dict:
    score = root_run.outputs.get("output") == example.outputs.get("golden_answer")
    return {"score": int(score), "key": "correct_label"}

In [12]:
from langsmith.evaluation import evaluate


results = evaluate(
    lambda inputs: get_sentiment(build_input_prompt(inputs["input"])),
    data=dataset_name,
    evaluators=[correct_label],
    experiment_prefix="Sentiment",
)

View the evaluation results for experiment: 'Sentiment-c9a77972' at:
https://smith.langchain.com/o/9c75e871-1666-51ff-aaa8-7c3e38165404/datasets/71866eb4-2615-43be-80da-05678ed760ec/compare?selectedSessions=e53447f1-4176-470d-ad66-2b6a96c92b9f




0it [00:00, ?it/s]